
In robotics, **inverse kinematics** refers to the problem of determining the set of joint variables (angles, in the case of revolute joints) required for a manipulator to reach a given position in space. This is one of the most fundamental problems in robot control, as tasks such as positioning, manipulation, and trajectory planning all require the ability to compute joint configurations corresponding to target positions.

The simplest nontrivial case arises in a **planar manipulator with two links**. Despite its apparent simplicity, the problem already illustrates the essential mathematical structure of inverse kinematics: nonlinear equations, multiple solutions, and possible singularities.


### Kinematic Model of the Two-Link Arm

Consider a robotic arm with:

* Two rigid links of lengths $L_1$ and $L_2$,
* Two revolute joints defined by angles $\theta_1$ (measured from the horizontal axis at the base) and $\theta_2$ (measured relative to the first link).

The position of the end-effector (the tip of the second link) in the Cartesian plane $(x, y)$ is given by the forward kinematics equations:

$$
x(\theta_1, \theta_2) = L_1 \cos(\theta_1) + L_2 \cos(\theta_1 + \theta_2)
$$

$$
y(\theta_1, \theta_2) = L_1 \sin(\theta_1) + L_2 \sin(\theta_1 + \theta_2)
$$

These equations map the joint space $(\theta_1, \theta_2)$ to the task space $(x, y)$.


### The Inverse Kinematics Problem

Given a desired end-effector position $(x_d, y_d)$, the inverse kinematics problem consists of finding values of $\theta_1$ and $\theta_2$ that satisfy:

$$
F_1(\theta_1, \theta_2) = x(\theta_1, \theta_2) - x_d = 0
$$

$$
F_2(\theta_1, \theta_2) = y(\theta_1, \theta_2) - y_d = 0
$$

Thus, the problem reduces to solving a system of two nonlinear equations with two unknowns.

#### Existence of the Solution

The target point $(x_d, y_d)$ is **reachable** if and only if it satisfies the following geometric constraint:

$$
|L_1 - L_2| \leq \sqrt{x_d^2 + y_d^2} \leq L_1 + L_2
$$

This ensures that the point lies within the annulus defined by the maximum and minimum reach of the arm.

Moreover, when the point is reachable, there generally exist **two distinct solutions**:

* The **elbow-up configuration**, where the joint bends outward,
* The **elbow-down configuration**, where the joint bends inward.

This non-uniqueness is a fundamental feature of inverse kinematics.

### Numerical Solution via Newton’s Method



Newton’s method is an **iterative numerical technique** used to approximate the roots of equations.

* For a **single equation** $f(x) = 0$, it generates successive approximations using the rule:

$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)} .
$$

Here, $f'(x_k)$ represents the derivative (slope) of the function at the current iterate. The method replaces the nonlinear curve by its tangent line and computes the intersection with the x-axis as the next approximation.

* For a **system of nonlinear equations**:

$$
F(\mathbf{x}) =
\begin{bmatrix}
F_1(x_1, x_2, \dots) \\
F_2(x_1, x_2, \dots) \\
\vdots
\end{bmatrix}
= \mathbf{0},
$$

Newton’s method generalizes to:

$$
\mathbf{x}_{k+1} = \mathbf{x}_k - J_F(\mathbf{x}_k)^{-1} F(\mathbf{x}_k),
$$

where $J_F(\mathbf{x}_k)$ is the **Jacobian matrix** of $F$ evaluated at $\mathbf{x}_k$.

In practice, instead of computing the inverse explicitly, one solves:

$$
J_F(\mathbf{x}_k) \cdot s^{(k)} = -F(\mathbf{x}_k),
$$

and updates:

$$
\mathbf{x}_{k+1} = \mathbf{x}_k + s^{(k)}.
$$


For the two-link planar manipulator, the forward kinematics are given by:

$$
x(\theta_1, \theta_2) = L_1 \cos(\theta_1) + L_2 \cos(\theta_1 + \theta_2),
$$

$$
y(\theta_1, \theta_2) = L_1 \sin(\theta_1) + L_2 \sin(\theta_1 + \theta_2).
$$

The inverse kinematics problem requires solving for $(\theta_1, \theta_2)$ such that the end effector reaches a prescribed target point $(x_d, y_d)$. This gives rise to the nonlinear system:

$$
F(\theta_1, \theta_2) =
\begin{bmatrix}
x(\theta_1, \theta_2) - x_d \\
y(\theta_1, \theta_2) - y_d
\end{bmatrix}
= \mathbf{0}.
$$

#### Newton Iteration for the Arm

Applying Newton’s method, the iterative scheme becomes:

$$
\begin{bmatrix}
\theta_1^{(k+1)} \\
\theta_2^{(k+1)}
\end{bmatrix}
=
\begin{bmatrix}
\theta_1^{(k)} \\
\theta_2^{(k)}
\end{bmatrix}
- J_F(\theta^{(k)})^{-1} F(\theta^{(k)}),
$$

with the Jacobian matrix:


$$
J_F(\theta_1,\theta_2) =
\begin{bmatrix}
\frac{\partial x}{\partial \theta_1} & \frac{\partial x}{\partial \theta_2} \\[2mm]
\frac{\partial y}{\partial \theta_1} & \frac{\partial y}{\partial \theta_2}
\end{bmatrix}
=
\begin{bmatrix}
- L_1\sin\theta_1 - L_2\sin(\theta_1+\theta_2) & -L_2\sin(\theta_1+\theta_2) \\[2mm]
\;\; L_1\cos\theta_1 + L_2\cos(\theta_1+\theta_2) & \;\; L_2\cos(\theta_1+\theta_2)
\end{bmatrix}.
$$

Thus, at each step of the method:

1. Evaluates the error vector

   $$
   F(\theta^{(k)}) =
   \begin{bmatrix}
   x(\theta_1^{(k)}, \theta_2^{(k)}) - x_d \\
   y(\theta_1^{(k)}, \theta_2^{(k)}) - y_d
   \end{bmatrix}.
   $$

2. Computes the Jacobian

   $$
   J_F(\theta^{(k)}) =
   \begin{bmatrix}
   - L_1 \sin(\theta_1^{(k)}) - L_2 \sin(\theta_1^{(k)} + \theta_2^{(k)}) &
   -L_2 \sin(\theta_1^{(k)} + \theta_2^{(k)}) \\
   L_1 \cos(\theta_1^{(k)}) + L_2 \cos(\theta_1^{(k)} + \theta_2^{(k)}) &
   L_2 \cos(\theta_1^{(k)} + \theta_2^{(k)})
   \end{bmatrix}.
   $$

3. Solves the linear system

   $$
   J_F(\theta^{(k)}) \cdot s^{(k)} = -F(\theta^{(k)}).
   $$

4. And, updates the joint angles

   $$
   \theta^{(k+1)} = \theta^{(k)} + s^{(k)}.
   $$

This process is repeated until the **end-effector error** satisfies:

$$
\|F(\theta^{(k)})\| < \varepsilon,
$$

where $\varepsilon$ is a predefined tolerance.

**Convergence Properties**

* **Quadratic convergence**: If the initial guess is sufficiently close to the solution and the Jacobian is nonsingular, the method converges very rapidly.
* **Sensitivity to initial guess**: Poor choices may lead to divergence.
* **Multiple solutions**: Nonlinear systems often admit more than one solution; the algorithm converges to the solution nearest to the initial guess.
* **Singularities**: If 
$$
\det(J_F) = L_1 L_2 \sin(\theta_2) = 0
$$
the iteration cannot proceed.

This occurs when $\theta_2 = 0$ (arm fully extended) or $\theta_2 = \pi$ (arm folded back on itself). At these points, the manipulator loses one degree of freedom in Cartesian space, and small changes in the end-effector position may require arbitrarily large changes in the joint angles.


### Python Implementation of Newton’s Method

Now, we can do a code implementation of Newton’s method for the inverse kinematics of the two-link planar robot arm. This implementation follows directly from the mathematical formulation presented earlier, and it will allow us to approximate the joint angles $\theta_1$ and $\theta_2$ that correspond to a desired target position $(x_d, y_d)$.

In [ ]:
import numpy as np

def forward_kinematics(theta, L1, L2):
    """Compute the (x, y) position of the end effector given joint angles."""
    theta1, theta2 = theta
    x = L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)
    y = L1 * np.sin(theta1) + L2 * np.sin(theta1 + theta2)
    return np.array([x, y])

def F(theta, L1, L2, xd, yd):
    """Nonlinear system: difference between current and desired position."""
    x, y = forward_kinematics(theta, L1, L2)
    return np.array([x - xd, y - yd])

def J(theta, L1, L2):
    """Jacobian matrix of the system."""
    theta1, theta2 = theta
    j11 = -L1 * np.sin(theta1) - L2 * np.sin(theta1 + theta2)
    j12 = -L2 * np.sin(theta1 + theta2)
    j21 = L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)
    j22 = L2 * np.cos(theta1 + theta2)
    return np.array([[j11, j12],
                     [j21, j22]])

def newton_inverse_kinematics(L1, L2, xd, yd, theta0, tol=1e-8, max_iter=100):
    """
    Solve the inverse kinematics problem using Newton’s method.

    Parameters
    ----------
    L1, L2 : float
        Lengths of the two links.
    xd, yd : float
        Desired target coordinates.
    theta0 : list or np.array
        Initial guess for [theta1, theta2].
    tol : float
        Tolerance for convergence (default: 1e-8).
    max_iter : int
        Maximum number of iterations allowed.

    Returns
    -------
    theta : np.array
        Approximated joint angles [theta1, theta2].
    k : int
        Number of iterations performed.
    """
    theta = np.array(theta0, dtype=float)
    for k in range(max_iter):
        f_val = F(theta, L1, L2, xd, yd)
        if np.linalg.norm(f_val) < tol:
            return theta, k
        J_val = J(theta, L1, L2)
        s = np.linalg.solve(J_val, -f_val)  # Solve J * s = -F
        theta = theta + s
    raise RuntimeError("Newton method did not converge within the maximum iterations")


### Numerical Experiments with Realistic Parameters

Having developed the Newton implementation, we now proceed to test the algorithm using realistic values for the link lengths and several reachable target points. This will illustrate both the practical behavior of the method and the importance of the initial guess.

**Choice of Link Lengths**

Let's choose:

$$
L_1 = 1.0 \, \text{m}, \quad L_2 = 0.8 \, \text{m}.
$$

* The **maximum reach** of the arm is $L_1 + L_2 = 1.8 \, \text{m}$.
* The **minimum reach** is $|L_1 - L_2| = 0.2 \, \text{m}$.

Thus, the reachable workspace is the annular region centered at the base, with radii $0.2 \, \text{m}$ and $1.8 \, \text{m}$.

**Selection of Target Points**

We now select three reachable target points within this workspace:

1. **Target A (near the center):**

   $$
   (x_d, y_d) = (0.5, 0.8)
   $$

2. **Target B (close to maximum extension):**

   $$
   (x_d, y_d) = (1.5, 0.3)
   $$

3. **Target C (intermediate position, elbow bent):**

   $$
   (x_d, y_d) = (0.2, 1.2)
   $$

Each of these points lies within the feasible workspace, as their distances from the origin satisfy:

$$
0.2 \leq \sqrt{x_d^2 + y_d^2} \leq 1.8.
$$



In [8]:
L1, L2 = 1.0, 0.8

# Three reachable target positions
targets = [
    ("Target A", (0.5, 0.8)),
    ("Target B", (1.5, 0.3)),
    ("Target C", (0.2, 1.2))
]

# Initial guess for joint angles
theta0 = [0.5, 0.5]

for name, (xd, yd) in targets:
    try:
        theta, iterations = newton_inverse_kinematics(L1, L2, xd, yd, theta0, tol=1e-8)
        pos = forward_kinematics(theta, L1, L2)
        print(f"{name}:")
        print("- Desired position:", (xd, yd))
        print("- Computed angles (theta1, theta2):", theta)
        print("- Reached position:", pos)
        print("- Iterations:", iterations, "\n")
    except RuntimeError as e:
        print(f"{name}: did not converge ({str(e)})")

Target A:
- Desired position: (0.5, 0.8)
- Computed angles (theta1, theta2): [0.16555028 2.05867147]
- Reached position: [0.5 0.8]
- Iterations: 6 

Target B:
- Desired position: (1.5, 0.3)
- Computed angles (theta1, theta2): [-0.2922011   1.11797973]
- Reached position: [1.5 0.3]
- Iterations: 6 

Target C:
- Desired position: (0.2, 1.2)
- Computed angles (theta1, theta2): [0.69239142 1.67096375]
- Reached position: [0.2 1.2]
- Iterations: 6 

